In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.sparse import diags
from scipy.optimize.linesearch import scalar_search_wolfe2

<ipython-input-1-dfbe500379d8>:5: DeprecationWarning: Please use `scalar_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import scalar_search_wolfe2


### Пункт № 1: градиентный спуск



> Класс, реализующий матричную-векторную функцию и действия с ней

In [13]:
class QuadraticOracle:
    def __init__(self, A, b):
        self.A = A
        self.b = b
    def func(self, x):
        return 0.5 * np.dot(self.A.dot(x), x) - self.b.dot(x)
    def grad(self, x):
        return self.A.dot(x) - self.b
    def funcDirectional(self, x, d, al):
        return np.squeeze(self.func(x + al * d))
    def gradDirectional(self, x, d, al):
        return np.squeeze(self.grad(x + al * d).dot(d))


> Градиентный спуск с шагом который можно задать самим (без ограничений на число итераций)

In [14]:
def gradientDescentConstantStep(oracle: QuadraticOracle, x_0, step, threshold=0.1):
    x_i = x_0
    func_x_i = oracle.func(x_i)
    grad_x_i = oracle.grad(x_i)
    a_k = None
    trace = []
    funcRes = []
    grad_x_0_norm = np.linalg.norm(grad_x_i)
    diff = 1e9
    funcRes.append(func_x_i)
    if x_0.size <= 2:
        trace.append(np.copy(x_i))
    count_of_appeals = 2
    grad_x_i_norm = np.linalg.norm(grad_x_i)
    while diff > threshold:
        d_k = -grad_x_i
        a_k = step
        x_i += a_k * d_k
        func_x_i = oracle.func(x_i)
        grad_x_i = oracle.grad(x_i)
        grad_x_i_norm = np.linalg.norm(grad_x_i)
        count_of_appeals += 3
        funcRes.append(func_x_i)
        diff = np.absolute(funcRes[-1] - funcRes[-2])
        if x_0.size <= 2:
            trace.append(np.copy(x_i))
    return x_i, trace, funcRes, count_of_appeals

Функционал для слежения за результатом градиетного спуска

In [15]:
b = np.array([0.0, 0.0])
oracle = QuadraticOracle(np.array([[1.0, 0.0], [0.0, 1.0]]), b)
x_i, trace, funcRes, count_of_appeals = gradientDescentConstantStep(oracle, np.array([4.0, 4.5]), 0.1, 0.01)
result = funcRes[-1]
print('число итераций: ' + str(len(funcRes)) + ', число обращений к оракулу: ' + str(count_of_appeals))

число итераций: 30, число обращений к оракулу: 89


Функция для отрисовки резултата ГС в пространстве



In [16]:
def draw_3D(trace, funcs):
    x_list, y_list = zip(*trace)
    l_list = funcs
    fig = plt.figure(figsize = (14,12))
    fig = plt.figure(figsize = (14,12))
    x = np.linspace(-5, 5, 1000)
    y = np.linspace(-5, 5, 1000)
    x, y = np.meshgrid(x, y)
    #f = 9 * x ** 2 + 12 * x * y + 9 * y ** 2 - 3 * x - 3 * y
    f = x**2 + y**2
    ax = fig.add_subplot(projection = '3d')
    ax.plot_surface(x, y, f, cmap = 'BuPu')
    ax.text(1.0, 2.0, 5.0, 'A', size = 25)
    ax.text(0, 0, 0, 'B', size = 25)
    ax.set_xlabel('x', fontsize = 15)
    ax.set_ylabel('y', fontsize = 15)
    ax.set_zlabel('f(x, y)', fontsize = 15)
    ax.plot(x_list, y_list, l_list, '.-', c = 'red')
    plt.show()

Функция для отрисовки линий уровня на плоскости соот-но

In [17]:
COLOR_RED = np.linspace(240, 166, 256) / 255.
COLOR_GREEN = np.linspace(244, 188, 256) / 255.
COLOR_BLUE = np.linspace(246, 203, 256) / 255.

In [18]:
def plotLevels(func):
    xrange = [-15.0, 15.0]
    yrange = [-15.0, 15.0]
    levels = [0, 0.25, 1, 4, 9, 16, 25, 36, 49, 64, 81]

    x = np.linspace(xrange[0], xrange[1], 100)
    y = np.linspace(yrange[0], yrange[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros(X.shape)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            Z[i, j] = func(np.array([X[i, j], Y[i, j]]))

    colors = np.vstack([COLOR_RED, COLOR_GREEN, COLOR_BLUE]).T
    my_cmap = ListedColormap(colors)

    _ = plt.contourf(X, Y, Z, levels=levels, cmap=my_cmap)
    CS = plt.contour(X, Y, Z, levels=levels, colors='#C79FEF')
    plt.clabel(CS, inline=1, fontsize=6, colors='#380282')
    plt.grid()

In [21]:
def plotTrace(trace):
    x_values, y_values = zip(*trace)
    plt.plot(x_values, y_values, '-o', linewidth=1.0, ms=5.0,
             alpha=1.0, c='C2', label="Trajectory")

In [ ]:
def three_d():
    b = np.array([3.0, 3.0])
    oracle = QuadraticOracle(np.array([[9.0, 6.0], [6.0, 9.0]]), b)
    x_i, trace, funcRes, count_of_appeals = gradientDescentConstantStep(oracle, np.array([-3.0, 3.0]), 0.1, 0.01)
    result = funcRes[-1]
    # print("результат: " + str(result))
    # print('число итераций: ' + str(len(funcRes)) + ', число обращений к оракулу: ' + str(count_of_appeals))
    
    draw_3D(trace, funcRes)
    
def two_d():
    b = np.array([3.0, 3.0])
    oracle = QuadraticOracle(np.array([[9.0, 6.0], [6.0, 9.0]]), b)
    x_i, trace, funcRes, count_of_appeals = gradientDescentConstantStep(oracle, np.array([-3.0, 3.0]), 0.1, 0.01)
    result = funcRes[-1]
    # print("результат: " + str(result))
    # print('число итераций: ' + str(len(funcRes)) + ', число обращений к оракулу: ' + str(count_of_appeals))
    plotLevels(oracle.func)
    plotTrace(trace)
    plt.show()

two_d()
three_d()

# Пункт 2: метод одномерного поиска

> Реализуйте метод одномерного поиска (метод дихотомии, метод Фибоначчи,метод золотого сечения) и градиентный спуск на его основе.

> Мы сделали градиентный спуск для функции n переменных, но все симметричные методы написаны для 2 переменных.

Метод золотого сечения


In [23]:
def goldenratio(x_i):
    e = 0.01
    a = -1
    b = 8
    x1 = a + ((3 - 5 ** 0.5) / 2) * (b - a)
    x2 = a + ((5 ** 0.5 - 1) / 2) * (b - a)
    
    fk = oracle.func(x_i)
    
    f1 = x1 * fk
    f2 = x2 * fk
    while abs(b - a) >= e:
        if f1 < f2:
            b, x2, f2 = x2, x1, f1
            x1 = a + ((3 - 5 ** 0.5) / 2) * (b - a)
            f1 = x1 * fk
        else:
            a, x1, f1 = x1, x2, f2
            x2 = a + ((5 ** 0.5 - 1) / 2) * (b - a)
            f2 = x2 * fk

    return (a + b) / 2, 1

> Градиентный спуск на основе метода дихотомии

In [24]:
def gradientDescentGoldenSearch(oracle: QuadraticOracle, x_0):
    x_i = x_0
    func_x_i = oracle.func(x_i)
    grad_x_i = -oracle.grad(x_i)
    trace = []
    funcRes = []
    funcRes.append(3)
    funcRes.append(func_x_i)
    if x_0.size <= 2:
        trace.append(np.copy(x_i))
    count_of_appeals = 2
    while abs(funcRes[-1] - funcRes[-2]) >= 0.01:
        d_k = -grad_x_i
        a_k, c = goldenratio(x_i)
        x_i += a_k * d_k
        func_x_i = oracle.func(x_i)
        grad_x_i = oracle.grad(x_i)
        count_of_appeals += 2 + c
        funcRes.append(func_x_i)
        if x_0.size <= 2:
            trace.append(np.copy(x_i))
    return x_i, trace, funcRes, count_of_appeals

In [ ]:
b = np.array([0.0, 0.0])
oracle = QuadraticOracle(np.array([[1.0, 0.0], [0.0, 1.0]]), b)
x_i, trace, funcRes, count_of_appeals = gradientDescentGoldenSearch(oracle, np.array([6.0, 6.0]))
result = funcRes[-1]
print("результат: " + str(result))
print('число итераций: ' + str(len(funcRes)) + ', число обращений к оракулу: ' + str(count_of_appeals))

plotLevels(oracle.func)
plotTrace(trace)
plt.show()

In [7]:
def weak_wolfe(oracle, x_i, d_k, method, c, c1, c2, al_0, previous_al=None):
      al = al_0 if previous_al is None else previous_al
      while (oracle.funcDirectional(x_i, d_k, al) > oracle.func(x_i) + c1 * al * oracle.gradDirectional(x_i, d_k, 0)):
        al /= 2
      return al
def strong_wolfe(oracle, x_i, d_k, method, c, c1, c2, al_0, previous_al=None):
      phi = lambda x: oracle.funcDirectional(x_i, d_k, x)
      derphi = lambda x: oracle.gradDirectional(x_i, d_k, x)
      phi_0 = phi(0)
      derphi_0 = derphi(0)
      al = scalar_search_wolfe2(phi=phi, derphi=derphi, phi0=phi_0, derphi0=derphi_0, c1=c1, c2=c2)[0]
      if al:
          return al
      else:
          return weak_wolfe(oracle, x_i, d_k, 'Armijo', c, c1, c2, al_0, previous_al)


In [4]:
def gradientDescent(oracle: QuadraticOracle, x_0, method, c0, c1, al, tolerance=1e-5, max_iter=1000):
    x_i = x_0
    func_x_i = oracle.func(x_i)
    grad_x_i = oracle.grad(x_i)
    a_k = None
    trace = []
    funcRes = []
    grad_x_0_norm = np.linalg.norm(grad_x_i)
    funcRes.append(func_x_i)
    if x_0.size <= 2:
        trace.append(np.copy(x_i))
    for _ in range(max_iter):
        d_k = -grad_x_i
        a_k = strong_wolfe(oracle, x_i, d_k, method, 0.5, c0, c1, al, 2 * a_k if a_k else None)
        if method == 'Division':
          a_k /= 2
        x_i += a_k * d_k
        func_x_i = oracle.func(x_i)
        grad_x_i = oracle.grad(x_i)
        grad_x_i_norm = np.linalg.norm(grad_x_i)

        funcRes.append(func_x_i)
        if x_0.size <= 2:
            trace.append(np.copy(x_i))
        if grad_x_i_norm ** 2 <= tolerance * grad_x_0_norm ** 2:
            return x_i, trace, funcRes
    return x_i, trace, funcRes

In [9]:
def condition(R, i, n, condition_numbers):
  for number in condition_numbers: 
    np.random.seed(1000 + i)
    diag = np.random.default_rng().uniform(low=1, high=number, size=n) 
    diag[0] = 1
    diag[-1] = number
    A = diags(diag).toarray()
    b = np.random.default_rng().uniform(low=1, high=number, size=n) 
    _, _, funcs = gradientDescent(QuadraticOracle(A, b), np.zeros(n), 'Wolfe', 0.1, 0.1, 0.2)
    R[n][i].append(len(funcs))
    # print('размерность: ' + str(n) + ', число обусловленности: ' + str(number) + ', количество итераций: ' + str(len(funcs)))
    # print(str(len(funcs)))

In [ ]:
def buildDeps():
    dims = [10, 100, 500, 1000]
    colors = ['c', 'm', 'y', 'r']
    condition_numbers = list(range(1, 1000, 50)) 
    number_of_samples = 4
    R = {}
    for n, color in zip(dims, colors):
        R[n] = [[] for _ in range(number_of_samples)] 
        print("размерность: " + str(n))
        for i in range(number_of_samples): 
            print('new function')
            condition(R, i, n, condition_numbers)
            plt.plot(condition_numbers, R[n][i], color=color, alpha=0.3, linestyle='--')
        plt.plot(condition_numbers, np.mean(R[n], axis=0), color=color, label='n = {}'.format(n))
    plt.legend()
    plt.grid()
    plt.ylabel('Количество итераций $R(n, \condition number)$')
    plt.xlabel('Число обусловленности $\conditiona number$')
buildDeps()
plt.show()